In [49]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from utils import *

In [51]:
df_merged = pd.read_csv('df_merged.csv')

/var/folders/mx/zvd2kjjx4z704zflxmthfp380000gn/T/ipykernel_97077/1377985770.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_merged = pd.read_csv('df_merged.csv')


In [54]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df_merged['genres_str'])

In [55]:
import numpy as np

def get_recommendations_by_weighted_rating(movie_title, df, tfidf_matrix, content_threshold=0.1, top_n=5):
    """
    Recommend movies that are both similar in content and have high weighted ratings.
    
    Parameters:
      movie_title (str): The title of the movie to base recommendations on.
      df (DataFrame): Merged DataFrame containing movies with 'primaryTitle', 'weightedRating', etc.
      tfidf_matrix (sparse matrix): TF-IDF feature matrix computed from movie features.
      content_threshold (float): A lower bound for content similarity to consider a movie similar.
      top_n (int): Number of recommendations to return.
      
    Returns:
      DataFrame: Recommended movies sorted by weighted rating.
    """
    # Find the movie index
    indices = df.index[df['primaryTitle'] == movie_title].tolist()
    if not indices:
        print("Movie not found.")
        return None
    movie_index = indices[0]
    
    # Get the query movie's TF-IDF vector
    query_vector = tfidf_matrix[movie_index]
    
    # Compute cosine similarities (assumes tfidf_matrix rows are L2-normalized)
    content_similarities = (tfidf_matrix @ query_vector.T).toarray().ravel()
    
    # Filter out movies that are not similar enough by content
    similar_mask = content_similarities >= content_threshold
    
    # Get candidate movies that pass the content similarity threshold (and exclude the queried movie)
    candidate_indices = np.where(similar_mask)[0]
    candidate_indices = candidate_indices[candidate_indices != movie_index]
    
    if candidate_indices.size == 0:
        print("No similar movies found based on content.")
        return None
    
    # For the candidate movies, extract the weighted ratings
    candidate_df = df.iloc[candidate_indices].copy()
    candidate_df['content_similarity'] = content_similarities[candidate_indices]
    
    # Now, re-rank candidates solely based on weightedRating (or a combination if you want)
    candidate_df = candidate_df.sort_values('weightedRating', ascending=False)
    
    # Return the top_n recommendations
    return candidate_df.head(top_n)[['primaryTitle', 'startYear', 'weightedRating']]

# Example usage:
recommendations = get_recommendations_by_weighted_rating("The Matrix", df_merged, tfidf_matrix, content_threshold=0.1, top_n=5)
print(recommendations)


                                      primaryTitle startYear  weightedRating
529021  Attack on Titan the Movie: The Last Attack      2024        9.007092
207815                             The Dark Knight      2008        8.999054
145777         Ramayana: The Legend of Prince Rama      1993        8.930293
307212                                   Inception      2010        8.799010
217663                                Interstellar      2014        8.698906
